In [66]:
import torch
from icecream import ic
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open('checkpoints/pendulum/goku_model.pkl', 'rb') as f:
    ckpt = torch.load(f, weights_only=False)
with open('data/pendulum/processed_data.pkl', 'rb') as processed_data_file:
    x = torch.load(processed_data_file, weights_only=False)
test_data = x['test']
train_data = x['train']
args = ckpt['args']   # saved argparse.Namespace
model_state = ckpt['model']
data_args = ckpt.get('data_args', None)


In [72]:
print(ckpt.keys())
print(x.keys())

ic| ckpt.keys(): dict_keys(['args', 'model', 'data_args', 'opt', 'epoch', 'best_val_loss'])
ic| x.keys(): dict_keys(['train', 'test'])


dict_keys(['train', 'test'])

In [67]:
# --- build model and load weights ---
print(train_data.shape)
print(test_data.shape)

ic| train_data.shape: (450, 100, 28, 28)
ic| test_data.shape: (50, 100, 28, 28)


(50, 100, 28, 28)

In [68]:
with open('data/pendulum/data_args.pkl', 'rb') as data_args:
    data_args = torch.load(data_args, weights_only=False)
ic(data_args)

ic| data_args: {'mask_rate': 0.01, 'model': 'pendulum', 'noise_std': 0.0}


{'mask_rate': 0.01, 'noise_std': 0.0, 'model': 'pendulum'}

In [69]:
import inspect
from models.GOKU import GOKU
from models.GOKU import create_goku_pendulum
sig = inspect.signature(create_goku_pendulum)
print(sig)
# parameter defaults (including kw-only)
spec = inspect.getfullargspec(create_goku_pendulum)
defaults = {}
if spec.defaults:
    defaults.update({name: val for name, val in zip(spec.args[-len(spec.defaults):], spec.defaults)})
if spec.kwonlydefaults:
    defaults.update(spec.kwonlydefaults)
print("defaults:", defaults)

ic| sig: <Signature (input_dim=[28, 28], latent_dim=16, rnn_input_dim=32, rnn_output_dim=16, ode_method='rk4', rnn_dropout=0.0, rnn_layers=2)>
ic| "defaults:": 'defaults:'
    defaults: {'input_dim': [28, 28],
               'latent_dim': 16,
               'ode_method': 'rk4',
               'rnn_dropout': 0.0,
               'rnn_input_dim': 32,
               'rnn_layers': 2,
               'rnn_output_dim': 16}


('defaults:',
 {'input_dim': [28, 28],
  'latent_dim': 16,
  'rnn_input_dim': 32,
  'rnn_output_dim': 16,
  'ode_method': 'rk4',
  'rnn_dropout': 0.0,
  'rnn_layers': 2})

In [73]:
import models.GOKU_pendulum as GOKU_pendulum
goku = GOKU(**defaults, encoder=GOKU_pendulum.Encoder, decoder=GOKU_pendulum.Decoder)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # change to whatever optimizer was used
model.load_state_dict(model_state)
optimizer.load_state_dict(ckpt['opt'])


In [71]:
ckpt["args"]

Namespace(num_epochs=10, mini_batch_size=64, seed=14, seq_len=50, delta_t=0.05, data_path='data/pendulum/', norm='zero_to_one', learning_rate=0.001, weight_decay=0.0, method='rk4', model='pendulum', kl_annealing_epochs=200, kl_start_af=1e-05, kl_end_af=1e-05, checkpoints_dir='checkpoints/pendulum/', cpu=False, grounding_loss=100.0)